In [ ]:
#IMPORT MODULES
from Bio import SeqIO
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import gzip
import sys
import os
from collections import defaultdict
from Bio.Blast import NCBIXML
from Bio.SeqFeature import SeqFeature
import subprocess
import gzip
import pypangraph as pp
from collections import Counter
import csv
import re

#IMPORT FILES
meta_data='/Users/jferrare/Documents/Good Lab Work/Gut Phage/uhgv_metadata.tsv.gz' #874051 rows
genomes='/Users/jferrare/Documents/Good Lab Work/Gut Phage/uhgv_mgv.fna.gz'
ani_reults='/Users/jferrare/Documents/Good Lab Work/Gut Phage/combined_uhgv_mgv_ani.txt' #30700847 rows
blast_results='/Users/jferrare/Documents/Good Lab Work/Gut Phage/combined_uhgv_mgv_blast_results.txt' #165831139 rows
ani_columns=['query_id', 'subject_id', 'num_alns', 'percent_identity', 'query_coverage', 'target_coverage']
blast_columns=['query_id','subject_id','percent_identity','length','mismatch','gapopen','qstart','qend','sstart','send','evalue','bitscore'] 
meta_columns=['uhgv_genome','uhgv_votu,votu_representative','original_study_alias','original_id','genome_length','is_jumbo_phage','gc_percent','checkv_quality','checkv_completeness','checkv_completeness_method','checkv_trimmed','viral_confidence','genomad_virus_score','genomad_virus_hallmarks','genomad_plasmid_hallmarks','viralverify_prediction','viralverify_score','checkv_viral_markers','checkv_host_markers','cds_count','cds_density','avg_cds_length','genetic_code',	'is_recoded','trna_count_total','trna_count_suppressor','sra_run','sra_sample','biosample','country	latitude','longitude']


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from Bio import SeqIO

def calculate_pairwise_heterozygosity(sequences):
    """Calculate pairwise heterozygosity for a set of sequences, excluding gap sites."""
    seq_array = np.array([list(seq) for seq in sequences], dtype='<U1')
    
    # Identify columns (sites) that contain at least one gap ('-')
    gap_mask = np.any(seq_array == '-', axis=0)
    
    # Remove columns with gaps
    filtered_seq_array = seq_array[:, ~gap_mask]
    
    # Get the number of sequences and ensure there's data left
    n_samples = filtered_seq_array.shape[0]
    if filtered_seq_array.shape[1] == 0:
        print("All sites contain gaps; skipping heterozygosity calculation.")
        return []

    # Compute pairwise heterozygosity
    heterozygosity_values = []
    for i in range(n_samples - 1):
        diffs = np.sum(filtered_seq_array[i] != filtered_seq_array[i + 1:], axis=1)
        heterozygosity_values.extend(diffs / filtered_seq_array.shape[1])  # Normalize by new site count

    return heterozygosity_values

def plot_heterozygosity_histogram(heterozygosity_values, output_path, votu_name=None):
    """Plot and save histogram of pairwise heterozygosity values."""
    plt.figure(figsize=(8, 6))
    plt.hist(heterozygosity_values, bins=25, color='blue', alpha=0.7, edgecolor='black')
    plt.xlabel("Pairwise Heterozygosity")
    plt.ylabel("Frequency")
    plt.xlim(0,.08)
    plt.title(f"Pairwise Heterozygosity Histogram{f' - {votu_name}' if votu_name else ''}")
    plt.savefig(output_path, dpi=300, bbox_inches='tight')
    plt.close()

def process_file(file_path, output_folder):
    """Process a single .fna file to compute heterozygosity and save the histogram."""
    sequences = [str(record.seq) for record in SeqIO.parse(file_path, "fasta")]
    
    if len(sequences) < 2:
        print(f"Skipping {file_path} - not enough sequences for pairwise comparison.")
        return
    
    heterozygosity_values = calculate_pairwise_heterozygosity(sequences)
    
    votu_name = os.path.basename(file_path).replace(".fna", "")
    output_path = os.path.join(output_folder, f"{votu_name}_heterozygosity.png")
    plot_heterozygosity_histogram(heterozygosity_values, output_path, votu_name)
    print(f"Saved histogram for {votu_name} to {output_path}")

def process_all_files_in_folder(folder_path, output_folder):
    """Iterate over all .fna files in a folder and process each."""
    os.makedirs(output_folder, exist_ok=True)
    
    for file_name in os.listdir(folder_path):
        if file_name.endswith(".fna"):
            file_path = os.path.join(folder_path, file_name)
            process_file(file_path, output_folder)


# Example usage
folder_path = "/Users/jferrare/Documents/Good Lab Work/Gut Phage/January_2025/large_vOTU_core_genome_alignments"
output_folder = "/Users/jferrare/Documents/Good Lab Work/Gut Phage/January_2025/pairwise_divergence_within_votu"
process_all_files_in_folder(folder_path, output_folder)